In [9]:
import tkinter as tk
from tkinter import messagebox, simpledialog
from pymongo import MongoClient
from bson.objectid import ObjectId
from datetime import datetime

# Connect to MongoDB (use your MongoDB connection string here)
client = MongoClient('mongodb://localhost:27017/')
db = client['library_db']
books_collection = db['books']
users_collection = db['users']

# Function for the Main Window
def main_window():
    root = tk.Tk()
    root.title("Pranay ka Library Management System")

    add_book_button = tk.Button(root, text="Add Book", command=add_book_window)
    add_book_button.grid(row=0, column=0, padx=20, pady=20)

    allot_book_button = tk.Button(root, text="Allot Book", command=allot_book_window)
    allot_book_button.grid(row=0, column=1, padx=20, pady=20)

    add_user_button = tk.Button(root, text="Add User", command=add_user_window)
    add_user_button.grid(row=0, column=2, padx=20, pady=20)

    root.mainloop()

# Function to display the Add Book Window
def add_book_window():
    add_book_win = tk.Toplevel()
    add_book_win.title("Add Book")

    tk.Label(add_book_win, text="Title").grid(row=0, column=0, padx=10, pady=5)
    title_entry = tk.Entry(add_book_win)
    title_entry.grid(row=0, column=1, padx=10, pady=5)

    tk.Label(add_book_win, text="Author").grid(row=1, column=0, padx=10, pady=5)
    author_entry = tk.Entry(add_book_win)
    author_entry.grid(row=1, column=1, padx=10, pady=5)

    tk.Label(add_book_win, text="Year").grid(row=2, column=0, padx=10, pady=5)
    year_entry = tk.Entry(add_book_win)
    year_entry.grid(row=2, column=1, padx=10, pady=5)

    tk.Label(add_book_win, text="ISBN").grid(row=3, column=0, padx=10, pady=5)
    isbn_entry = tk.Entry(add_book_win)
    isbn_entry.grid(row=3, column=1, padx=10, pady=5)

    def add_book():
        title = title_entry.get()
        author = author_entry.get()
        year = year_entry.get()
        isbn = isbn_entry.get()

        if not title or not author or not year or not isbn:
            messagebox.showerror("Input Error", "All fields are required.")
            return

        try:
            year = int(year)
        except ValueError:
            messagebox.showerror("Input Error", "Year must be a number.")
            return

        book = {
            "title": title,
            "author": author,
            "year": year,
            "isbn": isbn,
            "borrowed_by": None
        }
        books_collection.insert_one(book)
        messagebox.showinfo("Success", "Book added successfully.")
        add_book_win.destroy()

    add_button = tk.Button(add_book_win, text="Add Book", command=add_book)
    add_button.grid(row=4, column=0, columnspan=2, pady=10)

# Function to display the Allot Book Window
def allot_book_window():
    allot_book_win = tk.Toplevel()
    allot_book_win.title("Allot Book")

    tk.Label(allot_book_win, text="Select Book").grid(row=0, column=0, padx=10, pady=5)
    book_listbox = tk.Listbox(allot_book_win, width=50, height=10)
    book_listbox.grid(row=1, column=0, columnspan=2, padx=10, pady=5)

    books = books_collection.find({"borrowed_by": None})
    for book in books:
        book_listbox.insert(
            tk.END,
            f"{book['title']} by {book['author']} - {book['_id']}"
        )

    tk.Label(allot_book_win, text="User ID").grid(row=2, column=0, padx=10, pady=5)
    user_id_entry = tk.Entry(allot_book_win)
    user_id_entry.grid(row=2, column=1, padx=10, pady=5)

    def allot_book():
        try:
            selected_item = book_listbox.get(book_listbox.curselection())
            book_id = selected_item.split(" - ")[-1]

            user_id = user_id_entry.get()
            user = users_collection.find_one({"user_id": user_id})

            if not user:
                messagebox.showerror("User Not Found", "User ID not found.")
                return

            date_borrowed = datetime.now().strftime("%Y-%m-%d")
            books_collection.update_one(
                {"_id": ObjectId(book_id)},
                {"$set": {"borrowed_by": {"name": user['name'], "user_id": user_id, "date_borrowed": date_borrowed}}}
            )
            messagebox.showinfo("Success", f"Book allotted to {user['name']}.")
            allot_book_win.destroy()
        except Exception as e:
            messagebox.showerror("Selection Error", "Please select a book and enter a valid user ID.")

    allot_button = tk.Button(allot_book_win, text="Allot Book", command=allot_book)
    allot_button.grid(row=3, column=0, columnspan=2, pady=10)

# Function to display the Add User Window
def add_user_window():
    add_user_win = tk.Toplevel()
    add_user_win.title("Add User")

    tk.Label(add_user_win, text="User Name").grid(row=0, column=0, padx=10, pady=5)
    name_entry = tk.Entry(add_user_win)
    name_entry.grid(row=0, column=1, padx=10, pady=5)

    tk.Label(add_user_win, text="User ID").grid(row=1, column=0, padx=10, pady=5)
    user_id_entry = tk.Entry(add_user_win)
    user_id_entry.grid(row=1, column=1, padx=10, pady=5)

    def add_user():
        name = name_entry.get()
        user_id = user_id_entry.get()

        if not name or not user_id:
            messagebox.showerror("Input Error", "All fields are required.")
            return

        user = {"name": name, "user_id": user_id}
        users_collection.insert_one(user)
        messagebox.showinfo("Success", "User added successfully.")
        add_user_win.destroy()

    add_user_button = tk.Button(add_user_win, text="Add User", command=add_user)
    add_user_button.grid(row=2, column=0, columnspan=2, pady=10)

# Start the application
main_window()
